<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>

# Backpropagation Practice

## *Data Science Unit 4 Sprint 2 Assignment 2*

Implement a 3 input, 4 node hidden-layer, 1 output node Multilayer Perceptron on the following dataset:

| x1 | x2 | x3 | y |
|----|----|----|---|
| 0  | 0  | 1  | 0 |
| 0  | 1  | 1  | 1 |
| 1  | 0  | 1  | 1 |
| 0  | 1  | 0  | 1 |
| 1  | 0  | 0  | 1 |
| 1  | 1  | 1  | 0 |
| 0  | 0  | 0  | 0 |

If you look at the data you'll notice that the first two columns behave like an XOR gate while the last column is mostly just noise. Remember that creating an XOR gate was what the perceptron was criticized for not being able to learn. 

In [117]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error

In [194]:
df = pd.DataFrame({
    'x1':[0,0,1,0,1,1,0],
    'x2':[0,1,0,1,0,1,0],
    'x3':[1,1,1,0,0,1,0],
    'y':[0,1,1,1,1,0,0]},
    dtype='int8')
df

,x1,x2,x3,y
0,0,0,1,0
1,0,1,1,1
2,1,0,1,1
3,0,1,0,1
4,1,0,0,1
5,1,1,1,0
6,0,0,0,0


In [195]:
feats = list(df)[:-1]
target = 'y'
X = df[feats].values
y = np.array([[i] for i in df[target]])

In [196]:
X.shape, y.shape

((7, 3), (7, 1))

In [197]:
class NickNetwork:
    def __init__(self, inputs=3, hiddenlayers=4, outputnodes=1):
        self.inputs = inputs
        self.hiddenlayers = hiddenlayers
        self.outputnodes = outputnodes
    
    def sigmoid(self, x):
        return 1/(1 + np.exp(-x))
    
    def sigmoidprime(self, x):
        return x * (1 - x)
    
    def forward(self, X):
        self.hidden_sum = np.dot(X, self.weights1)
        self.activated_hidden = self.sigmoid(self.hidden_sum)
        self.output_sum = np.dot(self.activated_hidden, self.weights2)
        self.activated_output = self.sigmoid(self.output_sum)
        return self.activated_output
    
    def back(self, X, y, o):
        self.error = y - o
        self.delta = self.error * self.sigmoidprime(o)
        self.z2_error = self.delta.dot(self.weights2.T)
        self.z2_delta = self.z2_error * self.sigmoidprime(self.activated_hidden)
        self.weights1 += X.T.dot(self.z2_delta)
        self.weights2 += self.activated_hidden.T.dot(self.delta)
    
    def fit_predict(self, X, y, iter=1000, print_iters=100, verbose=False):
        '''
        Train network over 'iter' number of iterations
        print_iters = Print predicted information every 'print_iters' iterations when
        verbose is True
        verbose = Print information about model, adjust amount with 'print_iters'
        '''
        # Initialize weights
        self.weights1 = np.random.rand(self.inputs, self.hiddenlayers)
        self.weights2 = np.random.rand(self.hiddenlayers, self.outputnodes)

        if verbose:
            for i in range(iter):
                initial = self.forward(X)
                self.back(X,y,initial)
                if (i == 0) or (i % print_iters ==0):
                    print('=' * 9 + f'EPOCH {i}' + '=' * 9 + '\n')
                    print('Input: \n', X)
                    print('Actual Output: \n', y)
                    print('Predicted Output: \n', str(self.forward(X).round(2)))
                    print('Loss: \n', str(np.mean(np.square(y - self.forward(X)))))
        if verbose is False:
            for i in range(iter):
                initial = self.forward(X)
                self.back(X,y,initial)
            print('Number of iterations: \n', str(iter))
            print('Predicted Output: \n', str(self.forward(X).round(2)))

    # WAS WORKING THEN BROKE, NEED MORE TIME TO LOOK AT IT!    
    def score(self, X, y, iter=100, lines_to_show=10):
        '''
        Same as fit_predict method but returns an accuracy score instead
        '''
        self.weights1 = np.random.rand(self.inputs, self.hiddenlayers)
        self.weights2 = np.random.rand(self.hiddenlayers, self.outputnodes)

        for _ in range(iter):
            init = self.forward(X)
            self.back(X,y,init)
        df = pd.DataFrame(data=np.column_stack((y, self.activated_output)),columns=['Actual','Predicted'])
        print(f'For {iter} iterations, the MAE is: {mean_absolute_error(self.activated_output, y)}')
        print(df.head(lines_to_show))

In [208]:
nn = NickNetwork()

In [199]:
nn.fit_predict(X,y, iter=1)

Number of iterations: 
 1
Predicted Output: 
 [[0.73]
 [0.77]
 [0.76]
 [0.73]
 [0.73]
 [0.79]
 [0.68]]


In [200]:
nn.fit_predict(X,y,iter=1000, print_iters=250, verbose=True)

=========EPOCH 0=========

Input: 
 [[0 0 1]
 [0 1 1]
 [1 0 1]
 [0 1 0]
 [1 0 0]
 [1 1 1]
 [0 0 0]]
Actual Output: 
 [[0]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]]
Predicted Output: 
 [[0.71]
 [0.75]
 [0.73]
 [0.71]
 [0.69]
 [0.76]
 [0.66]]
Loss: 
 0.26112442052805995
=========EPOCH 250=========

Input: 
 [[0 0 1]
 [0 1 1]
 [1 0 1]
 [0 1 0]
 [1 0 0]
 [1 1 1]
 [0 0 0]]
Actual Output: 
 [[0]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]]
Predicted Output: 
 [[0.07]
 [0.78]
 [0.75]
 [0.81]
 [0.83]
 [0.36]
 [0.26]]
Loss: 
 0.0537513391952684
=========EPOCH 500=========

Input: 
 [[0 0 1]
 [0 1 1]
 [1 0 1]
 [0 1 0]
 [1 0 0]
 [1 1 1]
 [0 0 0]]
Actual Output: 
 [[0]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]]
Predicted Output: 
 [[0.02]
 [0.92]
 [0.91]
 [0.93]
 [0.93]
 [0.1 ]
 [0.14]]
Loss: 
 0.007864155564535171
=========EPOCH 750=========

Input: 
 [[0 0 1]
 [0 1 1]
 [1 0 1]
 [0 1 0]
 [1 0 0]
 [1 1 1]
 [0 0 0]]
Actual Output: 
 [[0]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]]
Predicted Output: 
 [[0.01]
 [0.94]
 [0.94]
 [0.95]
 [0.95]
 [0.06]

In [211]:
for i in [1, 100, 250, 500, 1000]:
    nn.score(X,y, i)

For 1 iterations, the MAE is: 0.4737394905330465
   Actual  Predicted
0     0.0   0.660843
1     1.0   0.685842
2     1.0   0.677450
3     1.0   0.656561
4     1.0   0.645881
5     0.0   0.697302
6     0.0   0.623765
For 100 iterations, the MAE is: 0.39465634432443275
   Actual  Predicted
0     0.0   0.297234
1     1.0   0.589750
2     1.0   0.574481
3     1.0   0.761852
4     1.0   0.758522
5     0.0   0.672377
6     0.0   0.477588
For 250 iterations, the MAE is: 0.2023315391648843
   Actual  Predicted
0     0.0   0.094168
1     1.0   0.789318
2     1.0   0.768016
3     1.0   0.814697
4     1.0   0.890211
5     0.0   0.317179
6     0.0   0.267215
For 500 iterations, the MAE is: 0.11257348165204403
   Actual  Predicted
0     0.0   0.028567
1     1.0   0.886331
2     1.0   0.887029
3     1.0   0.894186
4     1.0   0.895202
5     0.0   0.170652
6     0.0   0.151544
For 1000 iterations, the MAE is: 0.04732094583077221
   Actual  Predicted
0     0.0   0.008986
1     1.0   0.946958
2     1.

## Try building/training a more complex MLP on a bigger dataset.

Use the [MNIST dataset](http://yann.lecun.com/exdb/mnist/) to build the cannonical handwriting digit recognizer and see what kind of accuracy you can achieve. 

If you need inspiration, the internet is chalk-full of tutorials, but I want you to see how far you can get on your own first. I've linked to the original MNIST dataset above but it will probably be easier to download data through a neural network library. If you reference outside resources make sure you understand every line of code that you're using from other sources, and share with your fellow students helpful resources that you find.


### Parts
1. Gathering & Transforming the Data
2. Making MNIST a Binary Problem
3. Estimating your Neural Network (the part you focus on)

In [28]:
np.__version__

'1.17.3'

### Gathering the Data 

`keras` has a handy method to pull the mnist dataset for you. You'll notice that each observation is a 28x28 arrary which represents an image. Although most Neural Network frameworks can handle higher dimensional data, that is more overhead than necessary for us. We need to flatten the image to one long row which will be 784 values (28X28). Basically, you will be appending each row to one another to make on really long row. 

In [44]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

In [178]:
# input image dimensions
img_rows, img_cols = 28, 28

In [179]:
# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [180]:
x_train = x_train.reshape(x_train.shape[0], img_rows * img_cols)
x_test = x_test.reshape(x_test.shape[0], img_rows * img_cols)

# Normalize Our Data
x_train = x_train / 255
x_test = x_test / 255

In [181]:
# Now the data should be in a format you're more familiar with
x_train.shape

(60000, 784)

### Making MNIST a Binary Problem 
MNIST is multiclass classification problem; however we haven't covered all the necessary techniques to handle this yet. You would need to one-hot encode the target, use a different loss metric, and use softmax activations for the last layer. This is all stuff we'll cover later this week, but let us simply the problem for now: Zero or all else.

In [182]:
import numpy as np

y_temp = np.zeros(y_train.shape)
y_temp[np.where(y_train == 0.0)[0]] = 1
y_train = y_temp

y_temp = np.zeros(y_test.shape)
y_temp[np.where(y_test == 0.0)[0]] = 1
y_test = y_temp

In [183]:
# A Nice Binary target for ya to work with
y_train

array([0., 1., 0., ..., 0., 0., 0.])

### Estimating Your `net

In [184]:
sampler = np.random.randint(60000, size=1000)
sampler

array([49280, 49416,  1302, 42316, 21832, 36195, 32635, 30952, 44528,
       15170, 48289, 53513, 48845, 41928,  2501, 33355, 51091, 43858,
       27857, 25835, 48822, 58528, 42840, 19491,   323, 22765, 36211,
       48160, 33361, 39387, 13407, 32305, 22247, 21880, 21582, 29826,
        3797,  9752, 11767, 31033, 16577, 38640, 18558, 56520, 46895,
       33603, 43983, 13018, 42356, 57330, 17116, 25478, 45827, 19505,
       58646, 57953, 16426, 15972, 57333, 26438, 26847,   476, 53482,
       18585, 16787, 39026, 45693, 45796, 30000, 34141,  6687, 46541,
       57440, 34127, 28143, 48619, 16509, 59020, 33817,   132, 44018,
       23875, 59731, 49418, 44629, 53299, 33662, 47555, 15941, 38948,
       12547, 13922, 57578,  1180, 51761, 44018, 17995, 39477, 29402,
       54003, 57185, 19489, 58961, 53479, 13875, 59762, 21370, 50902,
        8347, 32685, 47600, 54578,  9208, 47341,  4878,  9279,  3184,
       35323, 15924, 44487, 50225, 31915, 42886, 35544,  9740,  3252,
        2383, 35482,

In [185]:
X = x_train[sampler]
y = np.array([[i] for i in y_train[sampler]])

In [186]:
X.shape, y.shape

((1000, 784), (1000, 1))

In [187]:
##### Your Code Here #####
nn = NickNetwork(inputs=784, hiddenlayers=3, outputnodes=1)

In [188]:
nn.fit_predict(X,y, iter=1000, print_iters=250, verbose=True)

=========EPOCH 0=========

Input: 
 [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
Actual Output: 
 [[1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [1.]
 [1.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]

This output is silly, let's create a new method in the class for this problem

In [189]:
nn = NickNetwork(inputs=784, outputnodes=1)

In [190]:
nn.score(X,y, iter=1000)

For 1000 iterations, the MAE is: 0.096
   Actual     Predicted
0     1.0  4.697925e-36
1     0.0  4.697925e-36
2     0.0  4.697925e-36
3     0.0  4.697925e-36
4     0.0  4.697925e-36
5     0.0  4.697925e-36
6     0.0  4.697925e-36
7     0.0  4.697925e-36
8     0.0  4.697925e-36
9     0.0  4.697925e-36


^ It thinks everything is a zero! ^

## Stretch Goals: 

- Make MNIST a multiclass problem using cross entropy & soft-max
- Implement Cross Validation model evaluation on your MNIST implementation 
- Research different [Gradient Descent Based Optimizers](https://keras.io/optimizers/)
 - [Siraj Raval the evolution of gradient descent](https://www.youtube.com/watch?v=nhqo0u1a6fw)
- Build a housing price estimation model using a neural network. How does its accuracy compare with the regression models that we fit earlier on in class?